In [1]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

import numpy as np
import pandas as pd
import pylab as pl
import math

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 
import matplotlib.mlab as mlab

In [2]:
df = pd.read_csv('car.csv',header=0)
df.head()

,buying,maint,doors,persons,lug_boot,safety,classe
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
##   Categorical Conversion

doors = {
            '2':'2',
            '3':'3',
            '4':'4',
            '5more':'5'
          }
persons = {
            '2':'2',
            '4':'4',
            'more':'5'
            }

classe = {
            'unacc':'0',
            'acc':'1',
            'good':'2',
            'vgood':'3'
        }

door_tmp = df['doors']
persons_tmp = df['persons']


X = df[['buying', 'maint', 'lug_boot', 'safety','classe']].values

ordinal_le = LabelEncoder()
X[:,0] = ordinal_le.fit_transform(X[:,0])
X[:,1] = ordinal_le.fit_transform(X[:,1])
X[:,2] = ordinal_le.fit_transform(X[:,2])
X[:,3] = ordinal_le.fit_transform(X[:,3])
X[:,4] = ordinal_le.fit_transform(X[:,4])

door_tmp = pd.get_dummies(door_tmp)
persons_tmp = pd.get_dummies(persons_tmp)


In [4]:
df = pd.DataFrame(X)
df.columns = ['buying', 'maint', 'lug_boot', 'safety', 'classe']
df[['doors_2','doors_3','doors_4','doors_5more']] = door_tmp
df[['persons_2','persons_4','persons_more']] = persons_tmp

# # Moving classe column to the end
classe_tmp = df['classe']
del df['classe']
df['classe'] = classe_tmp

df.head()

,buying,maint,lug_boot,safety,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more,classe
0,3,3,2,1,1,0,0,0,1,0,0,2
1,3,3,2,2,1,0,0,0,1,0,0,2
2,3,3,2,0,1,0,0,0,1,0,0,2
3,3,3,1,1,1,0,0,0,1,0,0,2
4,3,3,1,2,1,0,0,0,1,0,0,2


In [24]:
df.head().values

array([[3, 3, 2, 1, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 2, 2, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 2, 0, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 1, 2, 1, 0, 0, 0, 1, 0, 0, 2]], dtype=object)

In [11]:
print(df.describe())
print(df.classe.unique())

           doors_2      doors_3      doors_4  doors_5more    persons_2  \
count  1728.000000  1728.000000  1728.000000  1728.000000  1728.000000   
mean      0.250000     0.250000     0.250000     0.250000     0.333333   
std       0.433138     0.433138     0.433138     0.433138     0.471541   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.250000     0.250000     0.250000     0.250000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

         persons_4  persons_more  
count  1728.000000   1728.000000  
mean      0.333333      0.333333  
std       0.471541      0.471541  
min       0.000000      0.000000  
25%       0.000000      0.000000  
50%       0.000000      0.000000  
75%       1.000000      1.000000  
max       1.000000      1.000000  
[2 0 3 1]


In [12]:
# a=df['classe']
# plt.hist(a)
# plt.title('Classe (0, 1, 2, 3)')
# plt.show()

In [13]:
df.isnull().sum()

buying          0
maint           0
lug_boot        0
safety          0
doors_2         0
doors_3         0
doors_4         0
doors_5more     0
persons_2       0
persons_4       0
persons_more    0
classe          0
dtype: int64

In [14]:
label = df['classe']
# print(label)
features = df.drop('classe', 1)
features.head()

,buying,maint,lug_boot,safety,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more
0,3,3,2,1,1,0,0,0,1,0,0
1,3,3,2,2,1,0,0,0,1,0,0
2,3,3,2,0,1,0,0,0,1,0,0
3,3,3,1,1,1,0,0,0,1,0,0
4,3,3,1,2,1,0,0,0,1,0,0


In [15]:
# SVM

In [16]:

svm=SVC(C=1.0,kernel='rbf', max_iter=1000, tol=1e-05, verbose=0)
svm

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=1000, probability=False, random_state=None,
  shrinking=True, tol=1e-05, verbose=0)

In [17]:
X = np.array(features)
y = np.array(label).astype('int')

svm=svm.fit(X,y)



/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [18]:
print('dual coef \n', svm.dual_coef_)
print ('support vectors \n', svm.support_vectors_)
print('index of support vectors \n ', svm.support_)
print ('bias', svm.intercept_)
print('the classifier \n', svm)


dual coef 
 [[ 0.  0.  0. ..., -1. -1. -1.]
 [ 1.  1.  1. ..., -1. -1. -1.]
 [ 0.  0.  0. ..., -0. -0. -0.]]
support vectors 
 [[ 3.  2.  2. ...,  0.  1.  0.]
 [ 3.  2.  1. ...,  0.  1.  0.]
 [ 3.  2.  0. ...,  0.  1.  0.]
 ..., 
 [ 1.  1.  1. ...,  0.  0.  1.]
 [ 1.  1.  1. ...,  0.  1.  0.]
 [ 1.  1.  1. ...,  0.  0.  1.]]
index of support vectors 
  [ 227  230  232  233  239  241  242  254  257  259  260  263  265  266  268
  269  281  283  284  286  287  290  292  293  295  296  308  310  311  313
  314  317  319  320  322  323  335  338  340  341  347  349  350  362  365
  367  368  371  373  374  376  377  389  391  392  394  395  398  400  401
  403  404  416  418  419  421  422  425  427  428  430  431  551  554  556
  563  565  566  578  581  583  584  587  589  592  593  605  607  610  611
  614  616  619  620  632  634  637  638  641  643  646  647  659  662  664
  665  671  673  674  686  689  691  692  695  697  698  700  701  713  715
  716  718  719  722  724  725  727  

In [25]:
print(svm.predict([[3, 3, 2, 1, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 2, 2, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 2, 0, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 1, 2, 1, 0, 0, 0, 1, 0, 0]]))

[2 2 2 2 2]
